In [ ]:
import io
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm.notebook import tqdm

from helper import get_dominant_color
from constant import RGB, RYB

# database = pd.read_csv("database.csv")
out_name = "database_w_colors.csv"
color_RGB_col, color_RYB_col = "color_RGB", "color_RYB"
# database[color_RGB_col], database[color_RYB_col] = "", ""
# database.to_csv(out_name, index=False)
database = pd.read_csv(out_name)
start = database[color_RGB_col].isna().where(lambda x: x == True).first_valid_index()
data_root_path = Path.cwd().parent

progress = tqdm(total=21000)
progress.update(start)
i = start

def assign_colors(df, row_idx, pixel_access):
    df.at[row_idx, "color_RGB"] = get_dominant_color(RGB, pixel_access)
    df.at[row_idx, "color_RYB"] = get_dominant_color(RYB, pixel_access)

while i < 21000:
    with Image.open(path := data_root_path / Path(database.at[i, "path"])) as im:
        px = im.load()

    try:
        assign_colors(database, i, px)
    except IndexError:
        with open("errors.txt", "a") as errors:
            errors.write(str(path) + "\n")

        with open(path, "rb") as im:
            img = Image.open(io.BytesIO(im.read()), formats=("JPEG",))
            img.resize((224, 224)).save(path, "JPEG")

        with Image.open(path) as im:
            px = im.load()
            assign_colors(database, i, px)
    except Exception:
        with open("error.log", "w") as log:
            log.write(str(path))

        raise

    i += 1
    progress.update()

    if i % 100 == 0:
        database.to_csv(out_name, index=False)

database.to_csv(out_name, index=False)